# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [194]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

#from sklearn.linear_model import LogisticRegression
#import matplotlib.pyplot as plt
#from sklearn.cross_validation import train_test_split
#%matplotlib inline
#import sklearn.linear_model as lm


In [195]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [196]:
# frequency table for prestige and whether or not someone was admitted

df["prestige"].value_counts()


2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [197]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')
dummy_ranks

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
5,0.0,1.0,0.0,0.0
6,1.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0
8,0.0,0.0,1.0,0.0
9,0.0,1.0,0.0,0.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: To prevent perfect multicollinearity - the scenario when there is perfect correlation between variables and the model is perfectly singular - you need to leave one of the variables out from the new dummies. Intuitively, by specifying dummy variables, you are making your model into a comparative analysis of those different dummies (comparing them to each other). If you do not leave out one of the levels, it will be impossible to interpret the significance of the coefficients because there will be nothing to compare the results too; there is no baseline. Furthermore, the model will also be unable to run the regression unless a base category is set - or perhaps you could drop the intercept(?). Therefore, we need 3 variables.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [198]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummies.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [199]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(df.admit, dummy_ranks['prestige_1.0'])

prestige_1.0,0.0,1.0
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [ ]:
# Odds ratio if you attended prestige=1 ==> (243*33)/(93*28) = 3.08. You are 3.08 times more likely 
# to be admitted if you went to a presitious rank 1 school then if you did not.

# broken down, you can get to this by first calculating the odds that you of being admitted if
## you attended a #1 rank divided by odds of admission if you did not attend:

# admitted and did attend:
# (33/28)

#admitted and did not attend
#(243/98)

#OR = (33/28)*(243/98)==> 3.08. You could have just cross multiplied to get the same thing from the start.

#### 3.3 Calculate the odds ratio

In [ ]:
# 3.08

#### 3.4 Write this finding in a sentenance: 

Answer: You have a 3.08 more likely chance of getting admitted if you attended a number one prestitious school than if you did not.

#### 3.5 Print the cross tab for prestige_4

In [200]:
pd.crosstab(df.admit, dummy_ranks['prestige_4.0'])

prestige_4.0,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [141]:
OR = (12.*216.)/(114.*55.)
print OR

0.413397129187


#### 3.7 Write this finding in a sentence

Answer: If you attended a prestige 4 ranked school, you have a .4 odds of being admitted than those who attended prestige ranked 1,2,3 schools.

## Part 4. Analysis

In [303]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummies.ix[:, 'prestige_2':])
print data.head()

data.describe() # does not appear to have missing data. Outliers do not seem relevant to dataset. 

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           1.0           0.0
1      1  660.0  3.67           0.0           1.0           0.0
2      1  800.0  4.00           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           1.0


,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
count,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,0.372796,0.304786,0.168766
std,0.466044,115.717787,0.380208,0.484159,0.460898,0.375017
min,0.000000,220.000000,2.260000,0.000000,0.000000,0.000000
25%,0.000000,520.000000,3.130000,0.000000,0.000000,0.000000
50%,0.000000,580.000000,3.400000,0.000000,0.000000,0.000000
75%,1.000000,660.000000,3.670000,1.000000,1.000000,0.000000
max,1.000000,800.000000,4.000000,1.000000,1.000000,1.000000


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [304]:
# manually add the intercept
data['intercept'] = 1.0

In [305]:
data.describe()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
count,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.0
mean,0.317380,587.858942,3.392242,0.372796,0.304786,0.168766,1.0
std,0.466044,115.717787,0.380208,0.484159,0.460898,0.375017,0.0
min,0.000000,220.000000,2.260000,0.000000,0.000000,0.000000,1.0
25%,0.000000,520.000000,3.130000,0.000000,0.000000,0.000000,1.0
50%,0.000000,580.000000,3.400000,0.000000,0.000000,0.000000,1.0
75%,1.000000,660.000000,3.670000,1.000000,1.000000,0.000000,1.0
max,1.000000,800.000000,4.000000,1.000000,1.000000,1.000000,1.0


#### 4.1 Set the covariates to a variable called train_cols

In [300]:
train_cols = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0', 'intercept']
admit=data.admit

#### 4.2 Fit the model

In [218]:
sm.Logit?

In [301]:
logreg = sm.Logit(data['admit'], data[train_cols])
model_fit = logreg.fit() # Binary choice logit model

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


In [180]:
# getting a sense of covariates interacting with the target
#train_nd = data[data.admit == 0]
#train_d = data[data.admit == 1]

#plt.figure()
#plt.scatter(train_nd.gpa, train_nd.gre, alpha = .5, marker='+', c= 'b')
#plt.scatter(train_d.gpa, train_d.gre, marker='o', edgecolors = 'r', facecolors = 'none')
#plt.ylim([200,1000]); plt.xlim([2.0, 4.0])
#plt.legend( ('no admit', 'admit'), loc='upper right');

#### 4.3 Print the summary results

In [302]:
print model_fit.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 25 Jul 2016   Pseudo R-squ.:                 0.08166
Time:                        19:51:09   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [238]:
#np.exp?
print ('odd ratio of coeeficitions: %s' % np.exp(model_fit.params))

# odds ratio = p/1-p
# conf.columns = ['2.5%', '97.5%', 'OR'] ? Confidence intervals for coefficients in outputs above.

odd ratio of coeeficitions: gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


#### 4.5 Interpret the OR of Prestige_2

Answer: You have a .5 less likelihood of getting admitted compared with those in prestige_1.0

#### 4.6 Interpret the OR of GPA

Answer:  The odds of getting admitted increase as your GPA increases. The exact metric of that increase cannot be determined from the Odds Ratios.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [241]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [322]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [323]:
combos.head()

,0,1,2,3
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


#### 5.1 Recreate the dummy variables

In [336]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
new_dummy = pd.get_dummies(combos['prestige'], prefix='prestige', drop_first=True)

# keep only what we need for making predictions
cols_to_keep_1 = ['gre', 'gpa', 'prestige', 'intercept']
combos_1 = combos[cols_to_keep_1].join(new_dummy.ix[:, 'prestige_2':])

In [350]:
del combos_1['prestige']
combos_1.head()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,1.0,0.0,0.0,0.0
1,220.0,2.260000,1.0,1.0,0.0,0.0
2,220.0,2.260000,1.0,0.0,1.0,0.0
3,220.0,2.260000,1.0,0.0,0.0,1.0
4,220.0,2.453333,1.0,0.0,0.0,0.0


In [338]:
#train_cols1 = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0', 'intercept']
#admit=df.admit
#train_cols1

['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0', 'intercept']

#### 5.2 Make predictions on the enumerated dataset

In [349]:
model_fit = logreg.fit() 

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


In [355]:
combos_1['prediction'] = model_fit.predict(combos_1)

#logreg = sm.Logit(data['admit'], combos_1[train_cols1])
#model_fit_1 = logreg.fit() # Binary choice logit model

ValueError: shapes (400,7) and (6,) not aligned: 7 (dim 1) != 6 (dim 0)

In [356]:
combos_1

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,prediction
0,220.0,2.260000,1.0,0.0,0.0,0.0,0.827672
1,220.0,2.260000,1.0,1.0,0.0,0.0,0.557380
2,220.0,2.260000,1.0,0.0,1.0,0.0,0.503951
3,220.0,2.260000,1.0,0.0,0.0,1.0,0.090492
4,220.0,2.453333,1.0,0.0,0.0,0.0,0.848114
5,220.0,2.453333,1.0,1.0,0.0,0.0,0.594165
6,220.0,2.453333,1.0,0.0,1.0,0.0,0.541523
7,220.0,2.453333,1.0,0.0,0.0,1.0,0.103682
8,220.0,2.646667,1.0,0.0,0.0,0.0,0.866523
9,220.0,2.646667,1.0,1.0,0.0,0.0,0.629922


#### 5.3 Interpret findings for the last 4 observations

In [357]:
combos_1.tail()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,prediction
395,800.0,3.806667,1.0,0.0,0.0,1.0,0.545972
396,800.0,4.000000,1.0,0.0,0.0,0.0,0.985399
397,800.0,4.000000,1.0,1.0,0.0,0.0,0.946509
398,800.0,4.000000,1.0,0.0,1.0,0.0,0.934534
399,800.0,4.000000,1.0,0.0,0.0,1.0,0.582995


Answer: Someone applying to a less prestigious university has a lower probability of getting in. These are all people who have a perfect GRE score. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.